In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [3]:
# Run cell only once
!pip install -q xlrd
!git clone https://github.com/lucasace/cricket.git

Cloning into 'cricket'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 3489 (delta 5), reused 138 (delta 2), pack-reused 3333
Receiving objects: 100% (3489/3489), 18.88 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (3184/3184), done.


In [4]:
def predict_score(player, position, date): # position: batsman, all-rounder, wk, bowler
#date format yyyy-mm-dd
# When assigning position, make sure the file exists in github repo

  #Data Extraction
  dates = pd.read_csv('cricket/zip/' + player + '.csv')
  
  if position == 'all-rounder':
    batting_score = pd.read_csv('cricket/zip2/' + player + '.csv')
    bowling_score = pd.read_csv('cricket/bowl/' + player + '.csv')
    
    scores = batting_score.merge(bowling_score,how='left', left_on='match_id', right_on='Match_id')
    scores = dates.merge(scores,how='left', left_on='matchid', right_on='match_id')

    scores = scores[scores['date'] <= date]
    
    scores.drop(['Match_id', 'how dismissed','Wicket', 'Economy', 'Maidens', 'Unnamed: 0', 
                 'batting position', '6s', '4s', 'strike rate', 'match_id', 'date'], axis=1, inplace=True)

    scores['score'] = scores['score'].apply(lambda x : 0 if x == '-' else x)
    scores['score'] = scores['score'].apply(pd.to_numeric)
    scores['Score'] = scores['Score'].apply(pd.to_numeric)

    scores['Total'] = scores['score'] + scores['Score']
    scores.dropna(inplace=True)
    scores = scores[['Total']]    

    #print(scores[:5])
    y = list(scores['Total'])[::-1]
    #print(scores[:30])
    #return 
  elif position == 'batsman':
    batting_score = pd.read_csv('cricket/zip2/' + player + '.csv')
    scores = dates.merge(batting_score,how='left', left_on='matchid', right_on='match_id')
    scores.drop(['strike rate', '4s', '6s', 'how dismissed', 'batting position', 'matchid'], axis=1, inplace=True)

    scores.dropna(inplace=True)
    scores = scores[scores['score'] != '-']
    scores = scores[scores['date'] <= date]
    #print(scores[:5])
    y = list(scores['score'])[::-1]
    #print(scores[:30])
    #return
  elif position == 'wk':
    wk_score = pd.read_csv('cricket/wk/' + player + '.csv')
    scores = dates.merge(wk_score,how='left', left_on='matchid', right_on='MATCH_ID')
    scores.dropna(inplace=True)
    scores = scores[scores['date'] <= date]
    #print(scores[:5])
    y = list(scores['SCORE'])[::-1]
  else:
    bowling_score = pd.read_csv('cricket/bowl/' + player + '.csv')
    scores = dates.merge(bowling_score,how='left', left_on='matchid', right_on='Match_id')
    scores.drop(['Economy', 'Wicket', 'Maidens'], axis=1, inplace=True)
    scores.dropna(inplace=True)
    scores = scores[scores['date'] <= date]
    #print(scores[:5])
    y = list(scores['Score'])[::-1]

    #print(scores[:30])
    #return


  regr = LinearRegression(fit_intercept=True)

  for i in range(2):
    y_train = np.array(y[-5:]).reshape(-1,1)
    X_train = np.array(range(5)).reshape(-1,1)
    try:
      regr.fit(X_train, y_train)           
    except:
      return -1
    pred = regr.predict(np.array(5).reshape(1,-1))
    if pred[0][0] < 0:
      y.append(5)
    else:  
      y.append(pred[0][0])

  return y[-1]

In [15]:
names = pd.read_csv('cricket/players.csv')
names = list(names['Players'])
print(names[22], predict_score(names[22], 'batsman', '2019-06-09'))

Rohit Sharma  (India) 212.27999999999997


In [12]:
for i in range(337):
  if names[i] =='Rohit Sharma\xa0\xa0(India)':
    print(i)

113


In [7]:
names

['Faf du Plessis\xa0\xa0(South Africa)',
 'Ben Wheeler\xa0\xa0(New Zealand)',
 'David Warner\xa0\xa0(Australia)',
 'Hasan Ali\xa0\xa0(Pakistan)',
 'Shai Hope\xa0\xa0(West Indies)',
 'Hamid Hassan\xa0\xa0(Afghanistan)',
 'Abu Jayed\xa0\xa0(Bangladesh)',
 'Sharafuddin Ashraf\xa0\xa0(Afghanistan)',
 'Bjorn Fortuin\xa0\xa0(South Africa)',
 'Mirwais Ashraf\xa0\xa0(Afghanistan)',
 'Alex Carey\xa0\xa0(Australia)',
 'Shardul Thakur\xa0\xa0(India)',
 'Mohammed Siraj\xa0\xa0(India)',
 'Hayden Walsh\xa0\xa0(West Indies  United States)',
 'Sarfaraz Ahmed\xa0\xa0(Pakistan)',
 'Mohammad Amir\xa0\xa0(Pakistan)',
 'Billy Stanlake\xa0\xa0(Australia)',
 'M S Dhoni\xa0\xa0(India  ACC Asian XI)',
 'George Dockrell\xa0\xa0(Ireland)',
 'Glenn Maxwell\xa0\xa0(Australia)',
 'JJ Smuts\xa0\xa0(South Africa)',
 'Shuvagata Hom\xa0\xa0(Bangladesh)',
 'Rohit Sharma\xa0\xa0(India)',
 'Matt Parkinson\xa0\xa0(England)',
 'Asif Ali\xa0\xa0(Pakistan)',
 'Sam Billings\xa0\xa0(England)',
 'Tim Seifert\xa0\xa0(New Zealand)